<a href="https://colab.research.google.com/github/amydst/Data-Science-24/blob/main/Olympic_analysis_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [61.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,590 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd

In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [10]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/2/athlete_events.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("athlete_events.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

# Show DataFrame
df.show()




+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
| ID|                Name|Sex|Age|Height|Weight|          Team|NOC|      Games|Year|Season|       City|               Sport|               Event|Medal|
+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
|  1|           A Dijiang|  M| 24|   180|    80|         China|CHN|1992 Summer|1992|Summer|  Barcelona|          Basketball|Basketball Men's ...|   NA|
|  2|            A Lamusi|  M| 23|   170|    60|         China|CHN|2012 Summer|2012|Summer|     London|                Judo|Judo Men's Extra-...|   NA|
|  3| Gunnar Nielsen Aaby|  M| 24|    NA|    NA|       Denmark|DEN|1920 Summer|1920|Summer|  Antwerpen|            Football|Football Men's Fo...|   NA|
|  4|Edgar Lindenau Aabye|  M| 34|    NA|    NA|Denmark/Sweden|DEN|1900 Summer|1900|Summ

In [7]:

df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Games: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Season: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Event: string (nullable = true)
 |-- Medal: string (nullable = true)



In [11]:
#Create a temporary view.
df.createOrReplaceTempView('games')


In [12]:
# Get the number of distinct sports played in 2016

spark.sql("""
      SELECT COUNT(DISTINCT Sport)
      FROM games
      WHERE Year = 2016
""").show()

+---------------------+
|count(DISTINCT Sport)|
+---------------------+
|                   34|
+---------------------+



In [13]:
# Get the number of distinct sports played in 2012

spark.sql("""
      SELECT COUNT(DISTINCT Sport)
      FROM games
      WHERE Year = 2012
""").show()

+---------------------+
|count(DISTINCT Sport)|
+---------------------+
|                   32|
+---------------------+



In [19]:
# Write a subquery to get the distinct sports played in 2016 but not in 2012.

spark.sql("""
      SELECT DISTINCT Sport
      FROM games
      WHERE Year = 2016
      AND Sport NOT IN (
        SELECT DISTINCT Sport
        FROM games
        WHERE Year = 2012
      )
""").show()


+------------+
|       Sport|
+------------+
|        Golf|
|Rugby Sevens|
+------------+



In [22]:
# Another option could be using an EXCEPT
spark.sql("""
      SELECT DISTINCT Sport
      FROM games
      WHERE Year = 2016
      EXCEPT
        SELECT DISTINCT Sport
        FROM games
        WHERE Year = 2012

""").show()

+------------+
|       Sport|
+------------+
|        Golf|
|Rugby Sevens|
+------------+



In [23]:
# Get the year of the earliest Olympics in our data

spark.sql("""
      SELECT MIN(Year)
      FROM games
      """).show()


+---------+
|min(Year)|
+---------+
|     1896|
+---------+



In [24]:
# Get the sports that were played in the earliest Olympic games in our data
spark.sql("""
      SELECT DISTINCT Sport
      FROM games
      WHERE Year = 1896
""").show()




+-------------+
|        Sport|
+-------------+
|   Gymnastics|
|       Tennis|
|     Swimming|
|    Athletics|
|     Shooting|
|    Wrestling|
|Weightlifting|
|      Cycling|
|      Fencing|
+-------------+



In [25]:
# Determine which sports were in both the 1896 Olympics and the 2016 Olympics
spark.sql("""
      SELECT DISTINCT Sport
      FROM games
      WHERE Year = 2016
      AND Sport IN (
        SELECT DISTINCT Sport
        FROM games
        WHERE Year = 1896
      )
""").show()

+-------------+
|        Sport|
+-------------+
|   Gymnastics|
|       Tennis|
|     Swimming|
|    Athletics|
|     Shooting|
|    Wrestling|
|Weightlifting|
|      Cycling|
|      Fencing|
+-------------+



In [39]:
# Who are the top 5 Gold medal winners in the Olympics
top_gold = spark.sql("""
    SELECT Name, COUNT(medal) AS MedalCount, FIRST(medal) AS Medal
    FROM games
    WHERE medal = 'Gold'
    GROUP BY Name
    ORDER BY MedalCount DESC
    LIMIT 5
""")

top_gold.show()

+--------------------+----------+-----+
|                Name|MedalCount|Medal|
+--------------------+----------+-----+
|Michael Fred Phel...|        23| Gold|
|Raymond Clarence ...|        10| Gold|
|Frederick Carlton...|         9| Gold|
|Larysa Semenivna ...|         9| Gold|
|   Mark Andrew Spitz|         9| Gold|
+--------------------+----------+-----+



In [34]:
# Who are the top 5 Silver medal winners
spark.sql("""
    SELECT Name, COUNT(medal) AS MedalCount, FIRST(medal) AS Medal
    FROM games
    WHERE medal = 'Silver'
    GROUP BY Name
    ORDER BY MedalCount DESC
    LIMIT 5
""").show()

+--------------------+----------+------+
|                Name|MedalCount| Medal|
+--------------------+----------+------+
|Aleksandr Nikolay...|         6|Silver|
|Aleksandr Vladimi...|         6|Silver|
|Mikhail Yakovlevi...|         6|Silver|
|Shirley Frances B...|         6|Silver|
|           Yang Yang|         6|Silver|
+--------------------+----------+------+



In [35]:
# Who are the top 5 Bronze medal winners
spark.sql("""
    SELECT Name, COUNT(medal) AS MedalCount, FIRST(medal) AS Medal
    FROM games
    WHERE medal = 'Bronze'
    GROUP BY Name
    ORDER BY MedalCount DESC
    LIMIT 5
""").show()

+--------------------+----------+------+
|                Name|MedalCount| Medal|
+--------------------+----------+------+
|Aleksey Yuryevich...|         6|Bronze|
|Merlene Joyce Ott...|         6|Bronze|
|Franziska van Alm...|         6|Bronze|
|Harri Tapani Kirv...|         6|Bronze|
|Heikki Ilmari Sav...|         6|Bronze|
+--------------------+----------+------+



In [40]:
top_gold_pd = top_gold.toPandas()

In [41]:
import pandas as pd



top_gold_pd.to_csv('top5Gold.csv')
